## Компьютерное зрение в робототехнике | Домашнее задание 2

В этом домашнем задании нужно будет реализовать подсчет числа пальцев, которые показывает человек на видео fingers.mov из папки data.

Для удобства реализации значительная часть кода уже написана. Внимательно изнакомьтесь с заготовками, поймите, что происходит в коде и для чего нужна каждая конкретная функция.

Есть много вариантов, как можно подступиться к этой задаче. Приведем некоторые из них:

**Скелетонизация**
- получение маски
- обработка (фильтрация шумов, сглаживание)
- удаление всего, кроме самой большой связной компоненты
- скелетонизация (можно загуглить opencv skeletonization)
- нахождение кончиков пальцев (filter2d)
- фильтрация кончиков пальцев

**Дефекты выпуклости**
- получение маски
- обработка (фильтрация шумов, сглаживание)
- удаление всего, кроме самой большой связной компоненты
- нахождение контура, построение грубой аппроксимации
- нахождение дефектов выпуклости (convexity defects)

**Морфология**
- получение маски
- обработка (фильтрация шумов, сглаживание)
- удаление всего, кроме самой большой связной компоненты
- морфологические операции top hat/black hat

Можно выбрать любой из этих способов, который по душе, или придумать свой. Нейронные сети и другие методы машинного обучения использовать в решении нельзя :)

<h2 style="color:#A7BD3F;">Секция 1: подготовка</h2>

Этот код представляет из себя в сущности обертку над функцией обработки кадра. Завершение выполнения производится с помощью нажатия кнопки 'q' (работает только для английского языка).

In [1]:
import numpy as np
import cv2

In [2]:
class FrameProcessor:
    def __init__(self):
        pass
    
    def processing_loop(self, source, lth, hth, max_frame_num = -1,\
                        alternative_source="", save_to_file=""):
        i = 0
        results = []

        output_file = None
        
        #out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc('M','J','P','G'),
#                               30, (WINDX, WINDY))
#         out.write(canvas)
#         out.release()
        
        while (True):
            retval, frame = source.read()

            if (retval == False):
                print("Cannot read frame")
                
                if (alternative_source != ""):
                    print("Opening alternative source ", alternative_source)
                    source = cv2.VideoCapture(alternative_source)
                    continue
                
                else:
                    print("Exiting loop")
                    break

            result = self.process_frame(frame, lth, hth)
            results.append(result)
            key = cv2.waitKey(100) & 0xFF

            i += 1

            if (key == ord('q')):
                break
                        
            if (max_frame_num != -1 and i >= max_frame_num):
                break

        return results
    
    def process_frame(self, frame, lth, hth):
        return 5

<h2 style="color:#A7BD3F;">Секция 2: настройка цветового фильтра</h2>

Настройте параметры фильтрации по цвету. Это нужно делать в цветовом пространстве *HSV*. После этого запишите найденные параметры в *lth* и *hth*, их можно будет использовать позже.

In [3]:
# YOUR DEFAULT PARAMETERS BELOW
lth, hth = (0, 0, 0), (255, 255, 255)
# YOUR DEFAULT PARAMETERS ABOVE

class ColorFilterTuning(FrameProcessor):
    def __init__(self):
        super().__init__()
        
        cv2.namedWindow("color_filter_parameters")
                
        cv2.createTrackbar('rl', 'color_filter_parameters', lth[0], 255, self.nothing)
        cv2.createTrackbar('gl', 'color_filter_parameters', lth[1], 255, self.nothing)
        cv2.createTrackbar('bl', 'color_filter_parameters', lth[2], 255, self.nothing)
        cv2.createTrackbar('rh', 'color_filter_parameters', hth[0], 255, self.nothing)
        cv2.createTrackbar('gh', 'color_filter_parameters', hth[1], 255, self.nothing)
        cv2.createTrackbar('bh', 'color_filter_parameters', hth[2], 255, self.nothing)

    def nothing(self, inp):
        pass
    
    def process_frame(self, frame, lth, hth):
        hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        
        low_th =  (cv2.getTrackbarPos('rl', 'color_filter_parameters'),
                   cv2.getTrackbarPos('gl', 'color_filter_parameters'),
                   cv2.getTrackbarPos('bl', 'color_filter_parameters'))
        
        high_th = (cv2.getTrackbarPos('rh', 'color_filter_parameters'),
                   cv2.getTrackbarPos('gh', 'color_filter_parameters'),
                   cv2.getTrackbarPos('bh', 'color_filter_parameters'))
        
        mask = cv2.inRange(hsv, low_th, high_th)
        
        cv2.imshow("frame", frame)
        cv2.imshow("mask", mask)
        
        return (low_th, high_th)

In [4]:
cd

/home/zhanna


In [5]:
cd jupyter_env/cvr_course/data/videos

/home/zhanna/jupyter_env/cvr_course/data/videos


**Проверяем, что видео находится и открывается**

In [6]:
video_file = "fingers.mov"

cam = cv2.VideoCapture(video_file)
 
print(cam)
frame_offset = 100
cam.set(1, frame_offset)

tuner = ColorFilterTuning()

colors = tuner.processing_loop(cam, None, None, max_frame_num = -1,\
            alternative_source=video_file)
lth, hth = colors[-1]

print("Color filter parameters: ", lth, hth)
cam.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(100)

< cv2.VideoCapture 0x7fdd86b75550>


Cannot read frame
Opening alternative source  fingers.mov
Cannot read frame
Opening alternative source  fingers.mov
Cannot read frame
Opening alternative source  fingers.mov
Color filter parameters:  (71, 68, 133) (255, 255, 255)


-1

<h2 style="color:#A7BD3F;">Секция 3: подсчет пальцев</h2>

Реализуйте алгоритм нахождения числа пальцев, закомментировав функцию, которая всегда возвращает 3 и написав свою. Вы можете использовать *lth* и *hth*, которые получили выше. Заготовленные функции для обработки масок можно использовать, а можно и не использовать.

In [13]:
class FingersCounter(FrameProcessor):
    def __init__(self):
        super().__init__()

    def filter_cc(self, mask, area_th = -1):
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
        num_labels = output[0]
        labels = output[1]
        stats = output[2]
        #centroids = output[3]

        if (num_labels < 1):
            return mask
        
        if (area_th == -1):
            max_area = 1
            max_label = 1
            
            for i in range(1, num_labels):
                area = stats[i, cv2.CC_STAT_AREA]
                
                if (area > max_area):
                    max_area = area
                    max_label = i
            
            for i in range(1, len(stats)):
                if (i != max_label):
                    mask[np.where(labels == i)] = 0
                    
        else:
            for i in range(len(stats)):
                area = stats[i, cv2.CC_STAT_AREA]

                if (area < area_th):
                    mask[np.where(labels == i)] = 0

        return mask
    
    def fill_holes (self, img):
        (h, w) = img.shape

        before_area = img.sum ()

        img_enlarged = np.zeros ((h + 2, w + 2), np.uint8)
        img_enlarged [1:h+1, 1:w+1] = img

        img_enl_not = cv2.bitwise_not (img_enlarged)
        th, im_th = cv2.threshold (img_enl_not, 220, 255, cv2.THRESH_BINARY_INV);

        im_floodfill = im_th.copy()

        h, w = im_th.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)

        cv2.floodFill(im_floodfill, mask, (0,0), 255);
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

        result = im_out [1:h-1, 1:w-1]
 
        return result

     # YOUR CODE BELOW
    
    def calc_contour(self,k1,k2,msk):
        skeleton = self.skeletonize(msk)
        edge=cv2.filter2D(src=msk, ddepth=-1, kernel=k2)
        edge1=cv2.dilate(edge, k1, iterations=1)
        skel1 = cv2.dilate(skeleton, k1, iterations=1)
        x=cv2.bitwise_or(edge1, skel1)
        x=cv2.dilate(x, k1, iterations=2)
        x=cv2.erode(x,k1,iterations=2)
        x=cv2.bitwise_xor(x, msk)
        x=cv2.morphologyEx(x, cv2.MORPH_OPEN, k1)
        contours,_=cv2.findContours(x, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        return contours
    
    def process_frame(self, frame, lth, hth):
        perc = 5
        fingers_num = 0
        fltrd_frame = cv2.medianBlur(frame, perc) 
        hsv=cv2.cvtColor(fltrd_frame, cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(fltrd_frame, lth, hth) 
        offsets = ((200, -130), (350, -250))
        msk = mask[offsets[0][0]:offsets[0][1],offsets[1][0]:offsets[1][1]]
        msk = self.fill_holes(msk)
        k2 = np.array([[-1, -1, -1],
                            [-1, 8, -1],
                            [-1, -1, -1]])
        k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        contours = self.calc_contour(k1,k2,msk)      
        for i in  range(len(contours)):   
            p = cv2.arcLength(contours[i], True)
            s = cv2.contourArea(contours[i]) 
            if (p > 20):
                circ = 4*np.pi*s/(p**2)
                if circ > 0.57:
                    fingers_num += 1
                    
        return fingers_num
    
    
    def skeletonize(self, img): 
        size = np.size(img)
        skel = np.zeros(img.shape, np.uint8)

        element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))

        while True:
            open = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
            temp = cv2.subtract(img, open)
            eroded = cv2.erode(img, element)
            skel = cv2.bitwise_or(skel,temp)
            img = eroded.copy()
            
            if cv2.countNonZero(img)==0:
                break
                
        return skel
  # YOUR CODE ABOVE

In [14]:
cam = cv2.VideoCapture(video_file)
finger_counter = FingersCounter()
fingers_num = finger_counter.processing_loop(cam, lth, hth)

print(fingers_num)

cam.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(100)

Cannot read frame
Exiting loop
[4, 5, 0, 1, 0, 3, 5, 5, 0, 0, 1, 5, 5, 5, 5, 3, 4, 3, 4, 4, 1, 2, 2, 1, 1, 2, 2, 2, 2, 0, 1, 0, 1, 0, 2, 2, 2, 1, 2, 3, 3, 4, 4, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 3, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0]


-1

***

<h2 style="color:#A7BD3F;">Секция 4: оценивание</h2>

Решение, дающее значения метрики *accuracy* *0.5* и более, оцениваются полным баллом с линейной интерполяцией в области меньших значений.

Выполните эту клетку для получения своей оценки, изучите, как она была посчитана. Если в процессе выполнения задания появились вопросы или проблемы, пишите в чат или @elijahmipt в телеграме.

In [15]:
reference_fingers_num = [5, 5, 1, 0, 0, 5, 5, 5, 0, 0, 0, 5, 5, 5, 5, 4, 3, 3,\
                         3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2,\
                         2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,\
                         2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,\
                         2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1,\
                         3, 4, 0, 0, 0, 1]

max_grade = 100

corr_num = 0

for r, s in zip(reference_fingers_num, fingers_num):
    if (r == s):
        corr_num += 1

acc = corr_num / len(reference_fingers_num)

print("correct ", corr_num, " out of ", len(reference_fingers_num),
      corr_num / len(reference_fingers_num))

grade = min(acc * 2, 1) * max_grade

print("Your grade is ", "\033[92m{}\033[0m".format(str(int(grade)) +\
        " out of " + str(max_grade) + "; " + str(corr_num) + " frames out of "
        + str(len(reference_fingers_num))))

correct  60  out of  96 0.625
Your grade is  100 out of 100; 60 frames out of 96
